## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

[Placeholder for Binder link]

## Approach

   1. Identify available dates within a bounding box, which is also an area of interest (AOI) in this example, for a given collection
   2. Register a dynamic tiler search for an AOI and specific date range for a given collection
   3. 

## About the Data

TBD. Working with Ocean Net Primary Production (NPP) dataset

In [ ]:
import json
import requests

from folium import Map, TileLayer


In [ ]:
# import pystac_client

# from pystac_client import Client

# Endpoints
STAC_API_URL = "https://staging-stac.delta-backend.com"
RASTER_API_URL = "https://staging-raster.delta-backend.com"

# Declare collection of interest
collection_name = "MO_NPP_npp_vgpm"

In [ ]:
#Fetch STAC collection
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

In [ ]:
# Verify time density
collection["dashboard:time_density"]

In [ ]:
# Get collection summary
collection["summaries"]

Great, we can explore the year 2020 time series. Let's create a bounding box to explore the Venice Lagoon

In [ ]:
# Search criteria
# venice_bbox = [45.148479, 12.694402, 45.606375, 12.694402]
# venice_temporal_range = ["2020-01-01T00:00:00Z", "2020-12-31T00:00:00Z"] 

In [ ]:
# Venice AOI
venice_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
          [
            [
              12.099901459159526,
              45.60637513707738
            ],
            [
              12.099901459159526,
              45.14847949743654
            ],
            [
              12.694402360145261,
              45.14847949743654
            ],
            [
              12.694402360145261,
              45.60637513707738
            ],
            [
              12.099901459159526,
              45.60637513707738
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  

In [ ]:
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=100").json()["features"]
print(f"Found {len(items)} items")

This makes sense is our collection is monthly, so we should have 12 total items.

In [ ]:
items[0]

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium
m = Map(
    tiles="OpenStreetMap")

m.fit_bounds([[45.606375, 12.099901], [45.148479, 12.694402]]) 
folium.GeoJson(venice_aoi, name="Venice").add_to(m)
m

In [ ]:
# the bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics", 
        params={"url": item["assets"]["cog_default"]["href"]},
        json=geojson
    ).json()    
    return {
        **result["properties"], "start_datetime": item["properties"]["start_datetime"]
    }

In [ ]:
%%time 
stats = [generate_stats(item, venice_aoi) for item in items]

In [ ]:
stats[0]

In [ ]:
import pandas as pd

def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.1.", "") for col in df.columns]
    df["date"] = pd.to_datetime(df["start_datetime"])
    return df

df = clean_stats(stats)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20,10))

plt.plot(df["date"], df["mean"], 'black', label="Mean monthly Ocean NPP values")

plt.fill_between(df["date"], df["mean"] + df["std"], df["mean"] - df["std"], facecolor="lightgray", interpolate=False, label="+/- one standard devation")

plt.plot(df["date"], df["min"], color='blue', linestyle="-", linewidth=0.5, label="Max monthly NPP values")
plt.plot(df["date"], df["max"], color='red', linestyle="-", linewidth=0.5, label="Min monhtly NPP values")

plt.legend()
plt.title("Ocean NPP Values for Venice Lagoon, Italy (2020)")

In [ ]:
# !pip install stackstac cartopy 

These two packages weren't available in current `_env`

In [ ]:
# Looking at just a single image (one time stamp)
item = items[0]

In [ ]:
rescale_values = collection["summaries"]["cog_default"]
rescale_values

In [ ]:
tiles = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={item['collection']}&item={item['id']}"
    "&assets=cog_default"
    "&color_formula=gamma+r+1.05&colormap_name=rdbu_r"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
tiles

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium
m = Map(
    tiles="OpenStreetMap")

m.fit_bounds([[45.606375, 12.099901], [45.148479, 12.694402]]) 

map_layer = TileLayer(
    tiles=tiles["tiles"][0],
    attr="VEDA",
)

map_layer.add_to(m)
m

To Do: 
* Look at alignment of datasets 
* Fix color scale 